In [1]:
import os
import sys
import psycopg2
from sqlalchemy import create_engine
from io import StringIO
import pandas as pd
import csv
up_one =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(up_one)
up_two =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
sys.path.append(up_two)
from minorlab import metapdb
import subprocess

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
conn={}  # A dict of connections
cursor={}    # A dict of cursors
engine={}
def connect_db(db):
    print (f"     Connecting to {db['name']}...")
    try:
        #print (f"""    con_string is { db['con_string'] }""")
        exec (f"""conn[ "{db["name"]}" ] = psycopg2.connect("{db["con_string"]}")""" )
        exec (f"""cursor[ "{db['name']}" ] = conn[ "{db['name']}" ].cursor()""") # Pandas read_sql_query does not need a cursor
        exec (f"""engine["{db['name']}"] = create_engine( "{db['engine_string']}") """)
        print(f"  Connected to { db['name'] }.")
    except:
        print ('Check VPN. Not Connected.   <<<<===========#################')
connect_db(metapdb)
#connect_db(manuscripts)
print ('Connections done.')
# Run this cell to use single database functions
con = conn['metapdb']
cur = cursor['metapdb']
eng = engine['metapdb']

def df_to_empty_table(df, table,db='metapdb'):
    #create table using SLQalchemy 
    parts = table.split('.')
    if len(parts) ==1:
        schema='public'
        table_name=table
    else:
        schema, table_name = parts
    empty_df=  df.loc[df.iloc[:,0]=='Nonsense']
    try:
        empty_df.to_sql( schema=schema,name=table_name, con=engine[db], if_exists = 'replace', index=False)
        print (f'  Empty table {table} created for DataFrame with columns {empty_df.columns.to_list()}')
    except Exception as error:
        print (f'There was a problem with the SQLalchemy part. No table created.\n{error}')
        return 0
    return 1
    
def df_to_table(df, table,db='metapdb'):
    ''' Loads a DataFrame into a table. 
        If table is in the form schema.table, the schema will be used, otherwise schema="public".
        3rd ARG is databse; metapdb is default.'''
    if df_to_empty_table(df,table,db) == 0:
        return 0
    # save dataframe to an in memory buffer
    buffer = StringIO()
    
    df.to_csv(buffer, index=False, header=False,sep=';',quoting=csv.QUOTE_NONE,escapechar='\\')
    buffer.seek(0)
    
    # Try to copy from the buffer to the table
    try:
        print (f"""  Starting copy_from(buffer)""" )
        cursor[db].copy_from(buffer, table, sep=";",)
        conn[db].commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn[db].rollback()
        return 1
    print(f"df_to_table done for {table}.") 

     Connecting to metapdb...
  Connected to metapdb.
Connections done.


## Slight reformat of gencode information

In [79]:
with open ('Data/gencode.v43.pc_translations.fa', 'r') as f_in, open ('Data/gencode_v43_translations_mod.fa', 'w') as f_out:
    for line in f_in.readlines():
        if line[0] == ">":
            parts = line.split('|')
            f_out.write(f"{parts[0]}|{parts[5]}\n")
        else:
            f_out.write(line)

### Get list of human pdbs directly from PDB


In [ ]:
human_pdb= pd.read_csv ('Data/PDBe_human_entities.csv')
human_pdb_red = human_pdb.loc[:,['pdb_id','entity_id']]
df_to_table(human_pdb_red, 'temp_dcoop.human_pdb')

In [4]:
sql = """select ep.pdbid, ep.pdbx_strand_id as chains
            , ep.pdbx_seq_one_letter_code_can  as seq
        from temp_dcoop.human_pdb hp
        join pdbj.entity_poly ep on ep.pdbid = hp.pdb_id
            and hp.entity_id = cast(ep.entity_id as INT)
        where ep.type='polypeptide(L)'
        order by pdbid, pdbx_strand_id"""
df = pd.read_sql (sql,eng)
df
len (df['pdbid'].unique())

61321

## Write out as pdb_human_seqs.fa

In [ ]:
with open ('Data/pdb_human_seqs-230227.fa', 'w') as f:
    for i,parts in df.iterrows():
        chains = chains.replace(',','_')
        f.write(f">{parts['pdbid']}_{parts['chains']}\n{parts['seq']}\n")

## eliminate seqs < 30 AA and group by 100% identity

Group by reduces the list from ~97K to ~39K  
Removing chains less than 30AA removes 3167 


In [5]:
df2=df.copy()
df2['id']=df2['pdbid']+'_'+ df2['chains']
df2=df2.loc[:,['id','seq']]
df2 = df2.loc[ df2['seq'].str.len() >= 30 ]
print (f'df({len(df)}) - df2({len(df2)}) = {len(df) -  len(df2)} removed by 30 AA cutoff') 
df2.head()

df(96902) - df2(91217) = 5685 removed by 30 AA cutoff


,id,seq
0,"10gs_A,B",PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKAS...
1,"11gs_A,B",MPPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKA...
2,121p_A,MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...
3,12ca_A,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...
4,"12gs_A,B",MPPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKA...


In [6]:
groups = df2.groupby('seq')['id'].apply(list)
df2_g = groups.reset_index(name='chains')
print (f'df2({len(df2)}) - df2_g({len(df2_g)}) = {len(df2) -  len(df2_g)} less seqs after clustering at 100%') 
df2_g.head()

df2(91217) - df2_g(36104) = 55113 less seqs after clustering at 100%


,seq,chains
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,[6zm5_v]
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,[6g4s_k]
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[6zp4_X, 6zvj_Y, 7a09_X]"
3,AAAAAAAAAAAAAMKTIIALSYIFCLVFADYKDDDDKEKGISVPDH...,[7evw_R]
4,AAAAAAMIVQRVVLNSRPGKNGNPVAENFRMEEVYLPDNINEGQVQ...,"[2zb4_A, 2zb7_A, 2zb8_A]"


### get representative chain as for name and write out .fa

In [ ]:
df2_g['rep'] = [i[0] for i in df2_g['chains']]
with open ('Data/pdb_unique_human_seqs_230301.fa', 'w') as f:
    for i,parts in df2_g.iterrows():
        f.write(f">{parts['rep']}\n{parts['seq']}\n")

# Blast part
blastp -db ./a_gencode_fasta_same_prot_clustered.v35.fa -query ./uniprot_reviewed_canonical_and_isoform.fasta  
      -outfmt "6 qseqid qlen qstart qend sseqid slen sstart send evalue bitscore length nident gaps"   
      -out run_UN_to_GC_blast_result.tsv -num_threads 24 >> run_UN_to_GC_blast_result_stdout.txt 2>&1
makeblastdb -in <fasta file> -out <database name> -dbtype <type> -title <title> -parse_seqids
    
__[TEXT](https://www.ncbi.nlm.nih.gov/books/NBK279684/table/appendices.T.options_common_to_all_blast/)__

In [205]:
blast_path = "/home/daroc/ncbi-blast-2.13.0+/bin"
blastp_ = os.path.join (blast_path, 'blastp')
makedb = os.path.join (blast_path, 'makeblastdb')
s='gencode_v43_translations_mod.fa'
#q='one_pdb.fa'
q='some_pdbs.fa'
blast_options="-evalue 1e-20 -max_hsps 2"
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
column_str = " ".join(columns)
s_root = s.replace('.fa','')
outfmt = f"6 {column_str}"


In [202]:
#Project specific functions
# Make blast dbs
def makeblastdb (fasta):
    #  Note : This was problematic on OneDrive 
    root = fasta.replace('.fa','')
    cmd = f"""{makedb} -in {fasta} -out {root} -dbtype prot -title "Blastdb for {fasta}" -parse_seqids """
    print (cmd)
    db=subprocess.getoutput(cmd)
    if db[0:5]=="USAGE":
        print (f"Something went wrong with creation of the database\n{db}")
        
def blast_one(pdb, align=False):
    print (f'Will run blast for {pdb}')
    file = 'Data/pdb_unique_human_seqs_230301.fa'
    io_string = ""
    with open (file) as f:
        started = 0
        for line in f.read().splitlines():
            if pdb in line:
                started=1
                io_string += line + "\n"
            else: 
                if started ==1:
                    if ">" in line:
                        break
                    else:
                        io_string += line + "\n"
    if io_string:
        print (io_string)
        with open ('Data/temp.fa', 'w') as out:
            out.write(io_string)
    else:
        print (f'That PDB ({pdb}) not found.')
        return 0
    if align:
        blast_cmd = f'{blastp} -query "Data/temp.fa" -subject "Data/{s}" {blast_options}'
    else:
        blast_cmd = f'{blastp} -query "Data/temp.fa" -subject "Data/{s}" {blast_options} -outfmt "{outfmt}"'
    print (f"{blast_cmd}\n")
    blast_results=subprocess.getoutput(blast_cmd)
    if align:
        print (blast_results)
        return 0
    else:
        results_list = [line.split('\t') for line in blast_results.splitlines()]
        blast_df = pd.DataFrame(results_list, columns = columns)
        try:
            blast_df.to_excel('Data/blast_one.xlsx')
        except:
            print ("\n\nCouldn't write Excel file!!!")
        return (blast_df)

##### Make Databases in some cases

NOTE:  This will not work from OneDrive directory becuase of the spaces in the name. 

In [200]:
#  Should not need to edit below here 
###
makeblastdb(q)



/home/daroc/ncbi-blast-2.13.0+/bin/makeblastdb -in some_pdbs.fa -out some_pdbs -dbtype prot -title "Blastdb for some_pdbs.fa" -parse_seqids 


### Run blast with tabular output

In [ ]:
# DF output 
blast_cmd = f'{blastp} -query "Data/{q}" -subject "Data/{s}" {blast_options} -outfmt "{outfmt}"'
print (f"{blast_cmd}\n")
blast_results=subprocess.getoutput(blast_cmd)
results_list = [line.split('\t') for line in blast_results.splitlines()]
blast_df = pd.DataFrame(results_list, columns = columns)
try:
    blast_df.to_excel('Data/some_all.xlsx')
except:
    print ("\n\nCouldn't write Excel file!!!")
blast_df


/home/daroc/ncbi-blast-2.13.0+/bin/blastp -query "Data/some_pdbs.fa" -subject "Data/gencode_v43_translations_mod.fa" -evalue 1e-20 -max_hsps 2 -outfmt "6 qseqid qlen qstart qend sseqid slen sstart send evalue bitscore length nident gaps gapopen mismatch pident"



### The function blast_one will extract one PDB seq and run blast on it. 
USE: blast_one(pdb, [align=False] )
The default will give tabular result. align=True will give alignments



In [208]:
blast_df = blast_one('3iej_A,B')
blast_df

Will run blast for 3iej_A,B
>3iej_A,B
AALPDSVDWREKGCVTEVKYQGSCGASWAFSAVGALEAQLKLKTGKLVSLSAQNLVDCSTEKYGNKGCNGGFMTTAFQYI
IDNKGIDSDASYPYKAMDQKCQYDSKYRAATCSKYTELPYGREDVLKEAVANKGPVSVGVDARHPSFFLYRSGVYYEPSC
TQNVNHGVLVVGYGDLNGKEYWLVKNSWGHNFGEEGYIRMARNKGNHCGIASFPSYPEILQG

/home/daroc/ncbi-blast-2.13.0+/bin/blastp -query "Data/temp.fa" -subject "Data/gencode_v43_translations_mod.fa" -evalue 1e-20 -max_hsps 2 -outfmt "6 qseqid qlen qstart qend sseqid slen sstart send evalue bitscore length nident gaps gapopen mismatch pident"



,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident
0,"3iej_A,B",222,3,219,ENSP00000506248.1|CTSS-214,272,56,272,9.43e-164,453,217,216,0,0,1,99.539
1,"3iej_A,B",222,3,219,ENSP00000505359.1|CTSS-217,331,115,331,4.02e-163,454,217,216,0,0,1,99.539
2,"3iej_A,B",222,3,219,ENSP00000475557.2|CTSS-206,331,115,331,4.02e-163,454,217,216,0,0,1,99.539
3,"3iej_A,B",222,3,219,ENSP00000475176.2|CTSS-203,331,115,331,4.02e-163,454,217,216,0,0,1,99.539
4,"3iej_A,B",222,3,219,ENSP00000357981.3|CTSS-201,331,115,331,4.02e-163,454,217,216,0,0,1,99.539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,"3iej_A,B",222,1,200,ENSP00000217131.5|CTSZ-201,303,60,275,1.71e-23,97.1,226,75,36,10,115,33.186
143,"3iej_A,B",222,4,176,ENSP00000503676.1|CTSF-239,505,272,441,3.06e-22,95.5,174,63,5,3,106,36.207
144,"3iej_A,B",222,4,175,ENSP00000503238.1|CTSF-212,504,272,440,1.45e-21,93.6,173,62,5,3,106,35.838
145,"3iej_A,B",222,1,221,ENSP00000505285.1|CTSZ-219,314,60,301,2.08e-21,91.3,252,80,41,12,131,31.746


In [210]:
blast_df.to_excel('Data/3iej.xlsx')

In [203]:
blast_df = blast_one('7evw_R', True)
blast_df



Will run blast for 7evw_R
>7evw_R
AAAAAAAAAAAAAMKTIIALSYIFCLVFADYKDDDDKEKGISVPDHGFCQPISIPLCTDIAYNQTILPNLLGHTNQEDAG
LEVHQFYPLVKVQCSPELRFFLCSMYAPVCTVLDQAIPPCRSLCERARQGCEALMNKFGFQWPERLRCENFPVHGAGEIC
VGQNTSDGSGGPGGGPTAYPTAPYLPDLPFTALPPGASDGRGRPAFPFSCPRQLKVPPYLGYRFLGERDCGAPCEPGRAN
GLMYFKEEERRFARLWVGVWSVLCCASTLFTVLTYLVDMRRFSYPERPIIFLSGCYFMVAVAHVAGFLLEDRAVCVERFS
DDGYRTVAQGTKKEGCTILFMVLYFFGMASSIWWVILSLTWFLAAGMKWGHEAIEANSQYFHLAAWAVPAVKTITILAMG
QVDGDLLSGVCYVGLSSVDALRGFVLAPLFVYLFIGTSFLLAGFVSLFRIRTIMKHDGTKTEKLEKLMVRIGVFSVLYTV
PATIVLACYFYEQAFREHWERTWLLQTCKSYAVPCPPGHFPPMSPDFTVFMIKYLMTMIVGITTGFWIWSGKTLQSWRRF
YHRLSHSSKGETAVHHHHHHHHHHGLNDIFEAQKIEWHE

/home/daroc/ncbi-blast-2.13.0+/bin/blastp -query "Data/temp.fa" -subject "Data/gencode_v43_translations_mod.fa" -evalue 1e-20 -max_hsps 2

BLASTP 2.13.0+


Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro A.
Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J.
Lipman (1997), "Gapped BLAST and PSI-BLAST: a new generation of
protein dat

0

In [101]:
# For full results including alignments
blast_cmd = f'{blastp} -query "Data/{q}" -subject "Data/{s}"  {blast_options}'
print (f"{blast_cmd}\n")
blast_results=subprocess.getoutput(blast_cmd)
print (blast_results)

/home/daroc/ncbi-blast-2.13.0+/bin/blastp -query "Data/some_pdbs.fa" -subject "Data/gencode_v43_translations_mod.fa"  -evalue 1e-20 -max_hsps 2



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Criteria for 'at-length' match criteria
- length of query match equals the full length of the database entry
- no gaps
- percent matched > 95%

In [212]:
def at_length(row):
    if row['length'] == row['slen']:
        return 1
    else:
        return 0
def len_diff(row):
    return (int(row['slen']) - int(row['length']))
    
at_lengths = []
len_diffs = []

for i,row in blast_df.iterrows():
    at_lengths.append (at_length(row) )
    len_diffs.append(len_diff (row) )
blast_df['at_length']= at_lengths
blast_df['len_diff']= len_diffs
try:
    blast_df.to_excel('Data/check.xlsx')
except:
    print ("Couldn't write Excel file")
